# The Google Search Agent

The Workflow:

Search Tool: Find news/facts on a topic (e.g., "OpenAI's latest model release").

Date/Time Tool: Get current real-world time to ensure the news isn't outdated.

Validator Tool: A Python function that compares the search results against a "Trusted Sources" list or checks for logical consistency (e.g., "Is the release date in the future?").

The Loop: If the Validator finds a mismatch, Gemini must "Self-Correct" by searching again or clarifying the error.

In [1]:
from google import genai
from google.genai import types

In [16]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [3]:
import requests

In [17]:
import os 
key = os.getenv("GEMINI_API_KEY")
print(f"Current Key in memory: {key[:5]}...")

key2= os.getenv("SERPAPI_API_KEY")
print(f"Current Key in memory: {key2[:5]}...")


Current Key in memory: AIzaS...
Current Key in memory: def80...


In [4]:
from serpapi import GoogleSearch

In [22]:
def normalize_serpapi_results(results: list[dict]) -> list[dict]:
    """
    Converts SerpAPI organic results into a Gemini-friendly schema.
    """

    normalized = []

    for r in results:
        normalized.append({
            "title": r.get("title"),
            "url": r.get("link"),
            "source": r.get("source"),
            "date": r.get("date"),
            "summary": r.get("snippet"),
        })

    return normalized


In [24]:
def get_search_results(query: str):
    """
    Searches for the latest information on a topic using SerpAPI (Google Search).

    Args:
        query (str): Search query string.

    Returns:
        list[dict]: Normalized search results suitable for LLM tool consumption.
    """
    num_results = 1
    params = {
        "engine": "google",
        "q": query,
        "num": num_results,
        "hl": "en",
        "gl": "us",
        "api_key": os.getenv("SERPAPI_API_KEY"),
        "tbs" : "qdr:d"
    }

    search = GoogleSearch(params)
    results = search.get_dict()

    if "organic_results" not in results:
        return []

    serp_results=results.get("organic_results", [])
    final_results= normalize_serpapi_results(serp_results)
    return final_results
    

In [21]:
print(get_search_results("latest developments in generative AI"))

[{'position': 1, 'title': '10 Generative AI Trends In 2026 That Will Transform Work ...', 'link': 'https://bernardmarr.com/10-generative-ai-trends-in-2026-that-will-transform-work-and-life/', 'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://bernardmarr.com/10-generative-ai-trends-in-2026-that-will-transform-work-and-life/&ved=2ahUKEwibttmph9aRAxWEF2IAHQzVL28QFnoECCEQAQ', 'displayed_link': 'https://bernardmarr.com › 10-generative-ai-trends-in-2...', 'favicon': 'https://serpapi.com/searches/694bc5f280a078b3ab40ac44/images/61b66f7a6325739c9cb585bb43009a78c810e816e05e7d3ae0b18b9a441fbaac.png', 'date': 'Oct 28, 2025', 'snippet': '1. Generative Video Comes Of Age · 2. Authenticity Is King · 3. The Copyright Conundrum · 4. Agentic Chatbots—From Reactive To Proactive · 5.', 'snippet_highlighted_words': ['Generative Video Comes Of Age'], 'sitelinks': {'inline': [{'title': '10 Generative Ai Trends In...', 'link': 'https://bernardmarr.com/10-generative-a

In [ ]:
def validator_tool(info: str):
    """Checks if the provided info is consistent and verified."""
    if "2024" in info and "Gemini 2.0" in info:
        return {"status": "PASS", "message": "Information verified against internal database."}
    else:
        return {"status": "FAIL", "message": "Date mismatch detected. Please re-verify the release year."}

In [ ]:
available_tools = [get_search_results, validator_tool]

In [ ]:
# The secret sauce: Strict instructions on the process
system_instruction = """
You are a Verified Research Agent. 
Follow this strict protocol:
1. Search for information using get_search_results.
2. ALWAYS pass your findings to the validator_tool before answering.
3. If validator_tool returns 'FAIL', you MUST explain why and search again to find the correct data.
4. Only give the final answer to the user once the validator returns 'PASS'.
"""

client = genai.Client()

chat = client.chats.create(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        system_instruction=system_instruction,
        tools=available_tools,
        # 'automatic_function_calling' handles the loop for you in the SDK!
        automatic_function_calling=types.AutomaticFunctionCallingConfig(disable=False)
    )
)

response = chat.send_message("Tell me about Gemini 2.0 release.")
print(response.text)